In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from jupyterthemes import jtplot ; jtplot.style()
import hddm
from os import chdir ; chdir('/Users/albertwakhloo/Desktop/theoretical neuroscience/neuromatch/steinmetz full/five mice')

In [50]:
#neur = pd.read_csv('spikes.csv')
labelled_set = pd.read_csv('time_stamp_neurons_labels.csv')
stim = pd.read_csv('stimulus_series.csv')
labelled_set.head()

Unnamed: 0   spike_t area  maus  stim_t  neuron
0           0  0.003633  GPe     0     NaN   145.0
1           1  0.004967  BLA     0     NaN   245.0
2           2  0.005767  GPe     0     NaN   318.0
3           3  0.005900  BLA     0     NaN   537.0
4           4  0.006167  BLA     0     NaN   196.0

In [55]:
stim.stimtime.max()

997.9869514423416

In [175]:
def extract_trial_stim_avg(zone, stimulus_df, relevant_n, bins = .20) : 
    ''' 
    zone : process df containing maus, area, neuron, and spike time 
    relevant : neurons to slice out
    stimulus_df : mr clean df containing stim times 
    -------------------
    returns : df containing spike averages for a given time following stimulus onset'''
    full_df = pd.DataFrame()
    for mouse in zone.maus.unique() : 
        print(f'Generating mouse {mouse} spike rate data ...')
        mouse_df = pd.DataFrame()
        relevant_n_slice = zone.loc[(zone.maus == mouse) & 
                                    (zone.area.isin(relevant_n))] #grab any relevant neuron subset
        for area in relevant_n_slice.area.unique() :
            # grab a slice of the data containing a particular relevant area 
            area_subset = relevant_n_slice[relevant_n_slice.area == area]
            area_spike_set = np.array([])    
            for stim_time in stim.loc[stim.subj_idx == mouse].stimtime :
                #calculate trial by trial post stimulus spiking avg 
                n_spikes = len(
                    area_subset[(area_subset.spike_t <= stim_time + bins) &
                               (stim_time <= area_subset.spike_t)].spike_t.values)
                trial_avg = n_spikes / bins  
                area_spike_set = np.append(area_spike_set, trial_avg)
            mouse_df[area] = area_spike_set
            #print(area_spike_set.shape)
        mouse_df['maus'] = mouse 
        print(mouse)
        print(mouse_df.shape)
        full_df = full_df.append(mouse_df, ignore_index = True)
   # print('~~~ young success ~~~~')
    return full_df 

In [176]:
relevant_n = ['GPe', 'LH', 'MB', 'MOp', 'MOs', 'MRN', 'TH']
spk = extract_trial_stim_avg(labelled_set, stim, relevant_n)

Generating mouse 0 spike rate data ...
0
(140, 3)
Generating mouse 1 spike rate data ...
1
(118, 4)
Generating mouse 2 spike rate data ...
2
(153, 4)
Generating mouse 3 spike rate data ...
3
(150, 3)
Generating mouse 4 spike rate data ...
4
(151, 3)


In [177]:
print(spk[spk.maus == 1].shape)
stim[stim.subj_idx == 1].shape

(118, 6)


(118, 9)

In [178]:
spk.tail()

GPe     MB  maus  MRN  MOs      TH
707  NaN  500.0     4  NaN  NaN   480.0
708  NaN  820.0     4  NaN  NaN  1115.0
709  NaN  560.0     4  NaN  NaN   345.0
710  NaN  595.0     4  NaN  NaN   240.0
711  NaN  615.0     4  NaN  NaN   475.0

In [179]:
maus_0 = stim[stim.subj_idx == 0.].copy()
maus_0['striate'] = spk[spk.maus == 0].GPe.values
maus_0.head()

Unnamed: 0        rt  subj_idx  response  response_type  split_by  \
0           0  0.758448       0.0         1            1.0         1   
1           5  0.606752       0.0         0           -1.0         0   
2           8  1.094184       0.0         1            1.0         1   
3          12  0.660577       0.0         0           -1.0         0   
4          13  0.804386       0.0         0           -1.0         0   

   feedback  q_init   stimtime  striate  
0       1.0     0.7  38.905120   1340.0  
1       1.0     0.7  64.555633   1275.0  
2       1.0     0.7  76.900798   1165.0  
3       1.0     0.7  95.416796   1255.0  
4       1.0     0.7  98.989397   1295.0

In [ ]:
m = hddm.HDDMrlRegressor(maus_0, 'v ~ striate', include = 'alpha')
m.find_starting_values()
m.sample(3000, burn = 1500)

Adding these covariates:
['v_Intercept', 'v_striate']
-45.90793778549735
-45.907465805508096
 [-----------------66%-----             ] 1996 of 3000 complete in 570.8 sec

In [ ]:
def plot_posterior(param, trace, bins = 50) : 
    _, ax = plt.subplots(1, 2, figsize = (20, 7))
    x = np.arange(len(trace))
    ax[0].hist(trace[param], bins = bins)
    ax[1].plot(x, trace[param])
trace = m.get_group_traces()

In [ ]:
plot_posterior('v_Intercept', trace)

In [ ]:
plot_posterior('v_striate', trace)

In [ ]:
plot_posterior('a', trace)

In [ ]:
plot_posterior('t', trace)

In [ ]:
m.get_group_nodes()